# TCGA Pan-Cancer oncRNA Analysis

In [1]:
import pymongo
from pymongo import InsertOne
from pymongo import DeleteOne
import os
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import fdrcorrection
import scipy.stats as stats

## TCGA Cancer-Normal Filter
Here we remove RNA species that are present in less than 10% cancers across every cancer type.
We also remove RNA species that are present in more than 10% of normal tissue samples for at least one tissue type

In [2]:
tcga_metadata = pd.read_csv("metadata/TCGA_metadata.csv")
tcga_metadata = tcga_metadata.set_index("TCGAbarcode", drop=False)

normal_metadata = tcga_metadata[tcga_metadata["Sample"].str.contains("normal")]
normal_counts = normal_metadata.groupby("TCGA_abb", as_index=False)["Sample"].count()
#Only use normal tissue if at least 10 normal samples of that tissue for our 10% normal tissue samples filter.
normal_counts = normal_counts[normal_counts["Sample"] >= 10].set_index("TCGA_abb", drop=False) 
#This only includes normal tissue that have at least 10 samples.
normal_counts_dict = dict(zip(normal_counts["TCGA_abb"], normal_counts["Sample"]))

cancer_metadata = tcga_metadata[~tcga_metadata["Sample"].str.contains("normal")] #Does not contain normal.
cancer_counts = cancer_metadata.groupby("TCGA_abb", as_index=False)["Sample"].count()
cancer_counts_dict = dict(zip(cancer_counts["TCGA_abb"], cancer_counts["Sample"]))

In [3]:
normal_counts_dict

{'BLCA': 19,
 'BRCA': 104,
 'ESCA': 13,
 'HNSC': 44,
 'KICH': 25,
 'KIRC': 71,
 'KIRP': 34,
 'LIHC': 50,
 'LUAD': 46,
 'LUSC': 45,
 'PRAD': 52,
 'STAD': 45,
 'THCA': 59,
 'UCEC': 33}

In [4]:
def TCGA_filter(document, threshold=0.1):
    cancer_count = {}
    normal_count = {}
    samples = document["samples"]
    for s in samples:
        tissue = s["abb"] #Abbreviated tissue
        if "normal" in s["sample type"]:
            if tissue in normal_count:
                normal_count[tissue] += 1
            else:
                normal_count[tissue] = 1 
        else:
            if tissue in cancer_count:
                cancer_count[tissue] += 1
            else:
                cancer_count[tissue] = 1
    
    for tissue, count in normal_count.items():
        if tissue in normal_counts_dict: #Only apply filter if the tissue has at least 10 normal samples.
            total_count = normal_counts_dict[tissue] 
            if count/total_count > threshold: #We are only keeping RNAs that are present in less than or equal to 10% of normal samples for each tissue type.
                return False  
            
    for tissue, count in cancer_count.items():
        total_count = cancer_counts_dict[tissue]
        if count/total_count >= threshold: #We are keeping RNAs that are present in 10% or more of cancer samples for at least one tissue type.
            return True   
    return False

In [5]:
myclient=pymongo.MongoClient(port=27027)
mydb = myclient["TCGA_loci"]
aggcol = mydb["smRNAagg"]
fil_rnacol = mydb["smRNAfil"]

In [6]:
%%time
insert_req = []
cursor = aggcol.find(no_cursor_timeout=True)
batchcount = 0
batch = 1
for s in cursor:
    if TCGA_filter(s) == True:
        insert_req.append(InsertOne(s))
        batchcount += 1
    if batchcount == 10000:
        fil_rnacol.bulk_write(insert_req)
        insert_req = []
        batchcount = 0
        print(batch, "Done")
        batch += 1
if len(insert_req) > 0:
    fil_rnacol.bulk_write(insert_req)
    print(batch, "Done")

1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
20 Done
21 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
CPU times: user 36min 37s, sys: 3min 27s, total: 40min 4s
Wall time: 46min 6s


In [7]:
fil_rnacol.estimated_document_count()

275315

Post normal-cancer filter we have 275315 unique RNAs to then go through our Fisher Analysis.

In [8]:
myclient.close()

# Fisher Analysis
Following the initial filter, we then apply a Fisher Test for each RNA species to search for RNAs that are cancer-specific and not present in normal tissue (similar to the Nature Med paper: Cancer cells exploit an orphan RNA to drive metastatic progression, Fish et.al)

In [9]:
myclient=pymongo.MongoClient(port=27027)
mydb = myclient["TCGA_loci"]
fil_rnacol = mydb["smRNAfil"]

In [10]:
rna_cancer_sample_count = {}
for cancer in cancer_counts_dict.keys():
    rna_cancer_sample_count[cancer] = {}

In [11]:
cursor = fil_rnacol.find(no_cursor_timeout=True)
rna_norm_sample_count = {}
for rna in cursor:
    locus = rna["_id"]["locus"]
    samples = rna["samples"]
    rna_norm_sample_count[locus] = 0   
    for s in samples:
        if "normal" in s["sample type"]:
            rna_norm_sample_count[locus] += 1
        else:              
            tissue_abb = s["abb"]
            if locus in rna_cancer_sample_count[tissue_abb]:
                rna_cancer_sample_count[tissue_abb][locus] += 1
            else:
                rna_cancer_sample_count[tissue_abb][locus] = 1

In [12]:
myclient.close()

In [14]:
num_ctrl = 679 #All normals are pooled here at this step.
oncRNAs = set()
for cancer, data in rna_cancer_sample_count.items():
    num_cancer = cancer_counts_dict[cancer] #Total number of samples for a given CANCER
    cancer_locus_pval = {}
    for locus, test_count in data.items(): # DATA is a dictionary of loci as keys and corresponding counts of samples with the loci for a given CANCER as values 
        ctrl_count = rna_norm_sample_count[locus]
        oddsratio, pvalue = stats.fisher_exact([[test_count, ctrl_count],[num_cancer-test_count, num_ctrl-ctrl_count]], "greater")
        cancer_locus_pval[locus] = pvalue  
        
    cancer_locus_pval_df = pd.DataFrame.from_dict(cancer_locus_pval, orient="index", columns=["pval"])
    
    rej, fdr = fdrcorrection(cancer_locus_pval_df["pval"], alpha=0.1)
    cancer_locus_pval_df["fdr"] = fdr  
    fdr_cancer_locus_df = cancer_locus_pval_df[cancer_locus_pval_df["fdr"] <= 0.1] #fdr threshold of 0.1
    oncRNAs.update(fdr_cancer_locus_df.index)
    cancer_locus_pval_df.to_csv(f"results/fisher/{cancer}_fisher.csv")
    
len(oncRNAs)

275297

## Update Database

In [15]:
myclient=pymongo.MongoClient(port=27027)
mydb = myclient["TCGA_loci"]
fil_rnacol = mydb["smRNAfil"]
oncRNAcol = mydb["oncRNA"]

In [16]:
insert_req = []
cursor = fil_rnacol.find(no_cursor_timeout=True)
batchcount = 0
for s in cursor:
    locus = s["_id"]["locus"]
    if locus in oncRNAs:
        insert_req.append(InsertOne(s))
        batchcount += 1
    if batchcount == 10000:
        #Write to a new collection called oncRNA
        oncRNAcol.bulk_write(insert_req)
        insert_req = []
        batchcount = 0
if len(insert_req) > 0:
    oncRNAcol.bulk_write(insert_req)

In [17]:
oncRNAcol.estimated_document_count()

275297

## Save oncRNAs

In [18]:
myclient=pymongo.MongoClient(port=27027)
mydb = myclient["TCGA_loci"]
oncRNAcol = mydb["oncRNA"]

In [19]:
oncRNAs = []
cursor = oncRNAcol.find(no_cursor_timeout=True)
with open("results/pancancer_oncRNAs.bed", "w") as out:    
    for s in cursor:
        locus = s["_id"]["locus"]
        splits = locus.split(":")
        start = splits[1].split("-")[0]
        end = splits[1].split("-")[1]
        bed = f"{splits[0]}\t{start}\t{end}\t{locus}\t.\t{splits[2]}"
        out.write(bed + "\n")

# Done